In [ ]:
# neural net
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torchvision import datasets, transforms, models

from collections import OrderedDict

from tqdm import tqdm

import os

In [ ]:
# where is the data
list_dir = os.listdir('../input/cat-dog-data/Cat_Dog_data/')
list_dir

In [ ]:
# model for transfer learning
model = models.resnet18(pretrained=True)
model

In [ ]:
# process input images

data_dir = '../input/cat-dog-data/Cat_Dog_data'

train_transform = transforms.Compose([
    transforms.RandomRotation(30), 
    transforms.RandomResizedCrop(224), 
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], 
                        [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(255), 
    transforms.CenterCrop(224), 
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], 
                        [0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transform)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transform)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [ ]:
# freeze parameters so we do not backpropagate through them
for param in model.parameters():
    param.requires_grad = False
    
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(512, 256)), 
    ('relu', nn.ReLU()), 
    ('dropout', nn.Dropout(0.2)), 
    ('fc2', nn.Linear(256, 2)), 
    ('output', nn.LogSoftmax(dim=1))
]))

model.fc = classifier

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model.to(device)

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

In [ ]:
# training loop

epochs = 1
steps = 0 
running_loss = 0
print_every = 5

for epoch in tqdm(range(epochs)):
    
    for inputs, labels in trainloader:
        
        steps += 1
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        # clean gradient because they are accumulative
        optimizer.zero_grad()
        
        # forward pass
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        # save training loss
        running_loss += loss.item()
        
        if steps % print_every == 0:
            
            test_loss = 0
            accuracy = 0
            
            # model in evaluation mode to save computing time
            model.eval()
            
            with torch.no_grad():
                
                for inputs, labels in testloader:
                    
                    inputs, labels = inputs.to(device), labels.to(device)
                    
                    # compute predictions
                    logps = model.forward(inputs)
                    
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # compute accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = (top_class == labels.view(*top_class.shape))
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            
            # restart running_loss
            running_loss = 0
            
            # model to train mode 
            model.train()

In [ ]:
print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

In [ ]:
output_dir = '../working'
torch.save(model.state_dict(), output_dir + '/cat_dog_checkpoint.pth')